### *Импорты* ###

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

from keras.layers import Dropout
from keras.models import load_model

### *Загрузка файла* ###

In [2]:
data = pd.read_csv('../ann/../../db/basel(1940-2023).csv')

### *Отделение признаков и таргета в отдельные датафреймы* ###

In [3]:
# Необходимо удалить предсказываемые значения из датафрейма
features = data.drop(['Temperature [2 m elevation corrected]'], axis=1)

In [4]:
target = data['Temperature [2 m elevation corrected]']

### *Разделение на обучающий и тестовый наборы* ###

Данные не размешаны и не стратицированы для того, чтобы сохранить их структуру. Их размешивание и стратификация приведут к тому,
что модель плохо распознает ежемесячные и ежегожные погодные паттерны

In [5]:
# Обучающая часть
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, stratify=None, shuffle=False)

### *Стандартизация данных* ###

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Инициализация модели ####

In [7]:
model = keras.Sequential([
            layers.Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(32, activation='relu'),
            layers.BatchNormalization(),
            Dropout(0.2),

            layers.Dense(1),
        ])

#### Компиляция модели ####

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error')

#### Обучение модели ####

In [9]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=20)
# lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 10**(epoch / 20))

# model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])

#### Результативность модели ####

In [10]:
# prediction = model.predict(X_test_scaled)

# print('Mean Squared Error: {:.4f}'.format(mean_squared_error(y_test, prediction)))
# print('Mean Absolute Error: {:.4f}'.format(mean_absolute_error(y_test, prediction)))
# print('R-squared: {:.4f}'.format(r2_score(y_test, prediction)))

#### Сохранение модели ####

In [11]:
# model.save('best_model.keras')

#### Загрузка модели ####

In [12]:
model = load_model('best_model.keras')

prediction = model.predict(X_test_scaled)

print('Mean Squared Error: {:.4f}'.format(mean_squared_error(y_test, prediction)))
print('Mean Absolute Error: {:.4f}'.format(mean_absolute_error(y_test, prediction)))
print('R-squared: {:.4f}'.format(r2_score(y_test, prediction)))

2299/2299 [==============================] - 3s 1ms/step
Mean Squared Error: 0.9637
Mean Absolute Error: 0.7651
R-squared: 0.9859
